In [5]:
import pandas as pd
import numpy as np
import warnings
import re
import math
# warnings.filterwarnings("ignore")
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

In [36]:
chem_list = [
    u'C',
    u'Mn',
    u'Si',
    u'P',
    u'S',
    u'Cr',
    u'Ni',
    u'Cu',
    u'Al',
    u'V',
    u'Ti',
    u'Nb',
    u'Mo',
    u'N',
    u'B'
]

need_cols = chem_list + [
    '№ плавки', 'Поставщик'
]

In [37]:
def ch2f(x):
    if (type(x)==float) | (type(x)==int):
        x = str(x)
    if u'г' in x:
        print(x)
        x = x.replace(u'г.п.', '')
        x = x.replace(u'гп', '')
        x = str(x)
    if u'0.012. 0.014. 0.013' in str(x):
        x = x.replace(u'0.012. 0.014. 0.013', u'0.013')
    if x[-4:-2]=='e-':
        return float(x)
    elif '-' in str(x): 
#         print(x)
        x = x.replace(' ', '')
        x = str(x) + '0'
        tmp_map = list(map(float, str(x).replace(',', '.').split('-')))
        if (abs(tmp_map[-1]-tmp_map[-2])>0.05) | (tmp_map[-1]==0):
            return np.mean(tmp_map[:-1])
        else:
            return np.mean(tmp_map)
    else:
        return float(str(x).replace(',', '.'))

In [42]:
# подгрузка всех файлов испытаний
import os
dr = 'DATA/source/exp'
file_names = os.listdir(dr)

In [43]:
file_names.remove('prepared')

In [44]:
file_names

['1квартал2017.xls',
 '2012 1 квартал (январь, февраль, март)испр.xls',
 '2012 2 квартал (апрель, май, июнь)испр..xls',
 '2012 3 квартал (июль, август, сентябрь)испр..xls',
 '2012 4 квартал (октябрь, ноябрь, декабрь)испр.xls',
 '2013 1 квартал (январь, февраль, март)испр.xls',
 '2013 2 квартал (апрель, май, июнь)испр..xls',
 '2013 3 квартал (июль, август, сентябрь)испр..xls',
 '2013 4 квартал (октябрь, ноябрь, декабрь)испр.xls',
 '2014 1 квартал (январь, февраль, март)испр.xls',
 '2014 2 квартал (апрель, май, июнь)испр.xls',
 '2014 3 квартал (июль, август, сентябрь)испр.xls',
 '2014 4 квартал (октябрь, ноябрь, декабрь)испр.xls',
 '2015 1 квартал (январь, февраль, март)испр.xls',
 '2015 2 квартал (апрель, май, июнь)испр.xls',
 '2015 3 квартал ( июль, август, сентябрь)испр.xls',
 '2015 4 квартал (октябрь, ноябрь, декабрь)испр.xls',
 '2016 1 квартал (январь,февраль,март)испр.xls',
 '2016 2 квартал (апрель, май, июнь)испр.xls',
 '2016 3 квартал (июль, август, сентябрь)испр.xls',
 '2016 4 к

In [45]:
# файлы в словарь доступ по имени файла ({имя файла: словарь датафреймов файла} )
exp_files = {}
for name in file_names:
    exp_files[name] = pd.read_excel(dr+'\\'+name, None)
    print(name)

1квартал2017.xls
2012 1 квартал (январь, февраль, март)испр.xls
2012 2 квартал (апрель, май, июнь)испр..xls
2012 3 квартал (июль, август, сентябрь)испр..xls
2012 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2013 1 квартал (январь, февраль, март)испр.xls
2013 2 квартал (апрель, май, июнь)испр..xls
2013 3 квартал (июль, август, сентябрь)испр..xls
2013 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2014 1 квартал (январь, февраль, март)испр.xls
2014 2 квартал (апрель, май, июнь)испр.xls
2014 3 квартал (июль, август, сентябрь)испр.xls
2014 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2015 1 квартал (январь, февраль, март)испр.xls
2015 2 квартал (апрель, май, июнь)испр.xls
2015 3 квартал ( июль, август, сентябрь)испр.xls
2015 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2016 1 квартал (январь,февраль,март)испр.xls
2016 2 квартал (апрель, май, июнь)испр.xls
2016 3 квартал (июль, август, сентябрь)испр.xls
2016 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2квартал2017.xls
2квартал2017_2.xlsx
3 к

In [47]:
# номера плавок приводим к единому регистру (нижнему)
tmp = {}
for name in exp_files:
    for l in exp_files[name]:
        if '№ плавки' in exp_files[name][l]:
            exp_files[name][l]['№ плавки'] = exp_files[name][l]['№ плавки'].astype(str).str.lower()
        if '№ партии' in exp_files[name][l]:
            exp_files[name][l]['№ партии'] = exp_files[name][l]['№ партии'].astype(str).str.lower()
    tmp[name] = exp_files[name]
exp_files = tmp

In [50]:
# создаем единую таблицу с химией
chemistry = []
chemistry = exp_files['1квартал2017.xls'][u'Хим.анализ плавок (заготовка)']

for el in chem_list:
    try:
        chemistry[el]=chemistry[el].apply(ch2f).astype(float)
    except KeyError:
        print(el)    
# TODO : убрать этот список эта вкладка всегда первая!!!!!!!!!!!
"""TODO : убрать этот список эта вкладка всегда первая!!!!!!!!!!!"""
# TODO : убрать этот список эта вкладка всегда первая!!!!!!!!!!!
# TODO : убрать этот список эта вкладка всегда первая!!!!!!!!!!!

chemistry_list_names = [
    'Хим.анализ (заготовка)',
    'Хим.анализ плавок (заготовка)',
    'Хим.анализ плавок(заготовка)',
    'Хим.анализ плавок (Заготовка)',
    'Хим.анализ плавок (заготовки)',
    'Хим.анализ-заготовка',
    'Хим.анализ плавок'
]

s = 0
for name, exp_file in zip(exp_files, exp_files.values()):
    print(name)
    for list_chem in chemistry_list_names:
        if list_chem in exp_file:
            need_cols_spec = list(set(exp_file[list_chem].columns)&set(need_cols))
            exp_file[list_chem] = exp_file[list_chem][need_cols_spec]
            for el in chem_list:
                try:
                    exp_file[list_chem][el]=exp_file[list_chem][el].apply(ch2f).astype(float)
                except KeyError:
                    print("в файле нет данных о:", el)
            chemistry = pd.merge(chemistry, exp_file[list_chem], how = 'outer')
            s += exp_file[list_chem].shape[0]
            break

chemistry[u'№ плавки'] = chemistry[u'№ плавки'].astype(str)
chemistry.to_excel('chemistry.xlsx')

print(s)
print(chemistry.shape)

B
1квартал2017.xls
B
2012 1 квартал (январь, февраль, март)испр.xls
N
B
2012 2 квартал (апрель, май, июнь)испр..xls
B
2012 3 квартал (июль, август, сентябрь)испр..xls
2012 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2013 1 квартал (январь, февраль, март)испр.xls
Nb
Mo
B
2013 2 квартал (апрель, май, июнь)испр..xls
Nb
B
2013 3 квартал (июль, август, сентябрь)испр..xls
B
2013 4 квартал (октябрь, ноябрь, декабрь)испр.xls
Nb
B
2014 1 квартал (январь, февраль, март)испр.xls
Mo
B
2014 2 квартал (апрель, май, июнь)испр.xls
Mo
B
2014 3 квартал (июль, август, сентябрь)испр.xls
B
2014 4 квартал (октябрь, ноябрь, декабрь)испр.xls
Mo
B
2015 1 квартал (январь, февраль, март)испр.xls
B
2015 2 квартал (апрель, май, июнь)испр.xls
B
2015 3 квартал ( июль, август, сентябрь)испр.xls
Nb
B
2015 4 квартал (октябрь, ноябрь, декабрь)испр.xls
B
2016 1 квартал (январь,февраль,март)испр.xls
B
2016 2 квартал (апрель, май, июнь)испр.xls
B
2016 3 квартал (июль, август, сентябрь)испр.xls
B
2016 4 квартал (октябрь, н

In [51]:
# chemistry = chemistry[chemistry['Поставщик']=='ЭСПК']
print(chemistry.shape)

(13703, 17)


In [52]:
def get_more_chem(x):
    for el in x:
        for i in x[el]:
            if i!=None:
                x[el] = i
    return x
                
chemistry = chemistry.groupby('№ плавки').apply(get_more_chem)

In [53]:
chemistry.drop_duplicates(inplace=True)
chemistry.shape

(11347, 17)

In [54]:
# подгружаем режимы
mode = pd.read_excel('all_modes_clean.xlsx')
mode.shape
mode[u'№ плавки'] = mode[u'№ плавки'].astype(str)
mode.shape

(29255, 33)

In [55]:
# подгружаем режимы valid
mode_valid = pd.read_excel('valid_modes_clean.xlsx')
mode_valid[u'№ плавки'] = mode_valid[u'№ плавки'].astype(str)
mode_valid.shape

(1808, 22)

In [17]:
# TODO:  очищаем дубликаты по номеру партии (скорее всего какие-то косяки есть)
# mode[mode[['№ плавки', '№ партии']].duplicated(keep = False)].sort_values(['№ партии'])

In [56]:
merge_table = pd.merge(mode, chemistry, how ='left', on=['№ плавки'])
print(merge_table.shape)
merge_table_valid = pd.merge(mode_valid, chemistry, how ='left', on=['№ плавки'])
print(merge_table_valid.shape)

(29255, 49)
(1808, 38)


In [57]:
# Среднюю химию подгружаем 
# mean_chem_steel = pd.read_excel('mean_chem_steel.xlsx')

In [58]:
# merge_table.shape
# , mean_chem_steel.shape
1722 - 1698

24

In [59]:
# ls = merge_table.columns[:-15]
# merge_without_chem = merge_table[merge_table['C'].isnull()]
# print(merge_without_chem.shape)
# merge_without_chem = merge_without_chem[ls]
# merge_without_chem.shape

In [60]:
merge_table = merge_table[merge_table['C'].notnull()]
print(merge_table.shape)
merge_table_valid = merge_table_valid[merge_table_valid['C'].notnull()]
print(merge_table_valid.shape)

(28361, 49)
(1786, 38)


In [23]:
# merge_mean_chem = pd.merge(merge_without_chem, mean_chem_steel, how='left', on='марка стали')

In [24]:
# merge_mean_chem.shape

In [25]:
# merge_table = pd.merge(merge_table, merge_mean_chem, how ='outer')
# merge_table.shape

In [61]:
copy_merge_table = merge_table.copy()
copy_merge_table_valid = merge_table_valid.copy()
copy_merge_table.shape, copy_merge_table_valid.shape

((28361, 49), (1786, 38))

In [62]:
merge_table = copy_merge_table.copy()
merge_table_valid = copy_merge_table_valid.copy()
merge_table.shape, merge_table_valid.shape

((28361, 49), (1786, 38))

In [63]:
def clean_by_chem(df, mark, el, right_b, left_b=None):
    tmp = df.shape[0]
    if left_b==None:
        df = df[((df['марка стали']==mark) & ((df[el]<=right_b)|(pd.isnull(df[el]))))| (df['марка стали']!=mark)]
    else:
        df = df[((df['марка стали']==mark) & ((df[el]<=right_b)|(pd.isnull(df[el]))) & ((df[el]>=left_b)|(pd.isnull(df[el])))) | (df['марка стали']!=mark)]
#     print(df.shape)
    if (df.shape[0]-tmp!=0):
        print('удалено строк:', np.abs(df.shape[0]-tmp), 'причина', el, mark)
    return df

In [64]:
def clean_all_mark_chem(merge_table):
    # чистим 09Г2C
    merge_table = clean_by_chem(merge_table, '09Г2С', 'C', 0.14, 0.03)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Mn', 1.8, 1)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Si', 0.75, 0.45)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'S', 0.05)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'P', 0.05)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Al', 0.06, 0.015)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Ti', 0.035)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Ni', 0.3)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Cr', 0.3)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Cu', 0.3)
    # чистим 13ХФА
    merge_table = clean_by_chem(merge_table, '13ХФА', 'C', 0.19, 0.1)
    merge_table = clean_by_chem(merge_table, '13ХФА', 'Mn', 0.7, 0.3)
    merge_table = clean_by_chem(merge_table, '13ХФА', 'Si', 0.4, 0.13)
    merge_table = clean_by_chem(merge_table, '13ХФА', 'Cr', 0.8, 0.4)
    merge_table = clean_by_chem(merge_table, '13ХФА', 'V', 0.08, 0.02)
    # чистим 30Г2
    merge_table = clean_by_chem(merge_table, '30Г2', 'C', 0.35, 0.25)
    merge_table = clean_by_chem(merge_table, '30Г2', 'Mn', 1.6, 0.7)
    merge_table = clean_by_chem(merge_table, '30Г2', 'Si', 0.5, 0.13)
    # чистим 18ХМФБ
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'C', 0.25, 0.12)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'Mn', 0.7, 0.3)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'Si', 0.5, 0.13)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'Cr', 1.2, 0.45)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'Mo', 0.4, 0.1)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'V', 0.12, 0.02)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'Nb', 0.08, 0.015)
    # чистим 37Г2Ф
    merge_table = clean_by_chem(merge_table, '37Г2Ф', 'C', 0.45, 0.35)
    merge_table = clean_by_chem(merge_table, '37Г2Ф', 'Mn', 1.8, 0.7)
    merge_table = clean_by_chem(merge_table, '37Г2Ф', 'Si', 0.5, 0.13)
    merge_table = clean_by_chem(merge_table, '37Г2Ф', 'V', 0.12, 0.02)
    return merge_table

In [65]:
# merge_table[merge_table['№ партии']==203010]

In [66]:
merge_table = clean_all_mark_chem(merge_table)
merge_table.shape

удалено строк: 21 причина C 09Г2С
удалено строк: 1 причина Mn 09Г2С
удалено строк: 1 причина Si 09Г2С
удалено строк: 27 причина Al 09Г2С
удалено строк: 4 причина Ti 09Г2С
удалено строк: 28 причина C 13ХФА
удалено строк: 1 причина Cr 13ХФА
удалено строк: 73 причина C 30Г2
удалено строк: 7 причина C 18ХМФБ
удалено строк: 38 причина Mn 18ХМФБ
удалено строк: 3 причина Cr 18ХМФБ
удалено строк: 363 причина C 37Г2Ф
удалено строк: 15 причина Mn 37Г2Ф
удалено строк: 5 причина Si 37Г2Ф
удалено строк: 13 причина V 37Г2Ф


(27761, 49)

In [67]:
merge_table_valid = clean_all_mark_chem(merge_table_valid)
merge_table_valid.shape

удалено строк: 8 причина C 37Г2Ф


(1778, 38)

In [68]:
merge_table_valid.shape

(1778, 38)

In [70]:
merge_table.to_excel('DATA/prepared/merge_table_mean_chem.xlsx')

In [71]:
merge_table_valid.to_excel('DATA/prepared/valid_table_mean_chem.xlsx')